## Extracting CheckM lineages for each genomes

In [ ]:
import json
import pandas as pd
from pathlib import Path

In [ ]:
json_dir = "../../data/interim/checkm/bacillales_gbk/json"
json_files = [i for i in Path(json_dir).glob("*.json")]

In [ ]:
result = {}
json_file = json_files[0]
for num, json_file in enumerate(json_files):
    with open(json_file, "r") as f:
        data = json.load(f)
        lineage = data['marker lineage']
        genome = json_file.stem
        result[num] = {"genome_id" : genome,
                       "checkm_lineage" : lineage}
    

In [ ]:
df_checkm_lineages = pd.DataFrame.from_dict(result).T

In [ ]:
checkm_taxa = Path("../../resources/checkm/taxon_marker_sets.tsv", sep="\t")
checkm_taxa = pd.read_csv(checkm_taxa, sep="\t", header=None)

In [ ]:
checkm_taxa

In [ ]:
lineage_table = []
for taxa in df_checkm_lineages.checkm_lineage.unique():
    level, value = taxa.split("__")
    subset = checkm_taxa[(checkm_taxa[0].str.startswith(level)) & (checkm_taxa[1] == value)]
    lineage_table.append(subset)

In [ ]:
pd.concat(lineage_table)

In [ ]:
pd.read_csv("../../resources/checkm/genome_tree/genome_tree.metadata.tsv", sep="\t")

In [ ]:
# Read the file line by line
with open("../../data/checkm/checkm_table.txt", "r") as f:
    lines = f.readlines()

# Split each line using whitespace
data = [line.strip().split() for line in lines]

# Create a DataFrame from the split lines
df = pd.DataFrame(data)

# Assign column names
df.columns = ['genome_id', 'marker_lineage', 'UID', 'n_genomes', 'n_markers', 'n_marker_sets', '0', '1', '2', '3', '4', '5+', 'Completeness', 'Contamination', 'Strain heterogeneity']

df['UID'] = df['UID'].str.replace("(","")
df['UID'] = df['UID'].str.replace(")","")

In [ ]:
outfile = Path("../../results/tables/checkm_table.tsv")
outfile.parent.mkdir(parents=True, exist_ok=True)
df.to_csv(outfile, sep="\t", index=False)

In [ ]:
df_tree_checkm = pd.read_csv("../../resources/checkm/genome_tree/genome_tree.metadata.tsv", sep="\t")
outfile = Path("../../results/tables/checkm_reference.tsv")
outfile.parent.mkdir(parents=True, exist_ok=True)
df_tree_checkm[df_tree_checkm.UID.isin(df.UID)].to_csv(outfile, sep="\t", index=False)